In [2]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications.inception_v3 import InceptionV3
from tensorflow.keras.preprocessing import image
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Dropout
import tensorflow as tf
import pandas as pd
import numpy as np
import os
import time
import matplotlib.pyplot as plt
from sklearn.metrics import *
from sklearn.model_selection import train_test_split
from tensorflow.keras.callbacks import ReduceLROnPlateau
from tensorflow import keras
from tensorflow.keras.optimizers import SGD
import warnings
warnings.filterwarnings("ignore")

In [2]:
import splitfolders
splitfolders.ratio("Data", output="output",
    seed=1337, ratio=(.8, .2), group_prefix=None, move=False) # default values

Copying files: 3137 files [00:05, 606.18 files/s]


In [5]:
learning_rate = 0.0001
buffer_size = 519
batch_size = 16
epochs = 500
img_size = 224
key = 23

In [6]:
train_datagen = ImageDataGenerator(rescale=1. / 255,
                                   shear_range=0.2,
                                   zoom_range=0.2,
                                   horizontal_flip=True)

test_datagen = ImageDataGenerator(rescale=1. / 255)

training_set = train_datagen.flow_from_directory('output/train/',
                                                 target_size=(224,224),
                                                 batch_size=16)

test_set = test_datagen.flow_from_directory('output/val',
                                            target_size=(224,224),
                                            batch_size=16)

Found 2501 images belonging to 23 classes.
Found 636 images belonging to 23 classes.


In [7]:
def InceptionResNetV2_model():

    engine = tf.keras.applications.Xception(
        # Freezing the weights of the top layer in the InceptionResNetV2 pre-traiined model
        include_top = False,

        # Use Imagenet weights
        weights = 'imagenet',

        # Define input shape to 224x224x3
        input_shape = (img_size , img_size , 3),

    )

    x = tf.keras.layers.GlobalAveragePooling2D(name = 'avg_pool')(engine.output)
    x =Dropout(0.85)(x)
    out = tf.keras.layers.Dense(key, activation = 'softmax', name = 'dense_output')(x)


    # Build the Keras model

    model = tf.keras.models.Model(inputs = engine.input, outputs = out)
    # Compile the model

    model.compile(
        # Set optimizer to Adam(0.0001)
        optimizer = tf.keras.optimizers.Adam(learning_rate= 3e-4),
        #optimizer= SGD(lr=3e-4, momentum=0.9),
        #optimizer = tf.keras.optimizers.Nadam(learning_rate=3e-4, beta_1=0.9, 
                                            #  beta_2=0.999, epsilon=1e-07, name="Nadam"),
        # Set loss to binary crossentropy
        #loss = tf.keras.losses.sparse_categorical_crossentropy,
        loss = tf.keras.losses.CategoricalCrossentropy(),
        # Set metrics to accuracy
        metrics = ['accuracy']
    )

    return model

In [8]:
early_stopping = tf.keras.callbacks.EarlyStopping(monitor='loss', patience=3)
learning_rate_reduction = keras.callbacks.ReduceLROnPlateau(monitor='val_accuracy',
                                                            patience=2,
                                                            verbose=2,
                                                            factor=0.5,
                                                            min_lr=0.00001)
reduce_lr =  keras.callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.5,
                              patience=3, min_lr=0.00001)

In [9]:
def train():
    time_start = time.time()
    
    model = InceptionResNetV2_model()
    
    model.summary()
    history = model.fit(training_set, epochs= epochs , verbose = 2, validation_data = test_set,
                       callbacks=[early_stopping, reduce_lr , learning_rate_reduction])
    
    

    model.save_weights('layer22-wskindiseases1.h5')
    model.save('layer22-skindiseases1.h5')
    
    print('Model saved.')
    
    time_end = time.time()
    print('Training Time:', time_end - time_start)
    print('\n')

    return history


In [10]:
def test():
    #test_labels = np.array(test_labels)

    from tensorflow import keras
    print('Testing:')
    mod = InceptionResNetV2_model()
    mod.load_weights('Layer22-wskindiseases1.h5')
    mod.evaluate(test_set)
    
    #prob = mod.predict(test_set)
    #predIdxs = np.argmax(prob, axis=1) 


    print('\n')
    #print(classification_report(test_set.labels, predIdxs,target_names = key, digits=5))
    return predIdxs, prob, mod 

In [13]:
if __name__ == "__main__":
    
    train_history = train() 
    predIdxs,prob,  model = test()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 224, 224, 3) 0                                            
__________________________________________________________________________________________________
block1_conv1 (Conv2D)           (None, 111, 111, 32) 864         input_1[0][0]                    
__________________________________________________________________________________________________
block1_conv1_bn (BatchNormaliza (None, 111, 111, 32) 128         block1_conv1[0][0]               
__________________________________________________________________________________________________
block1_conv1_act (Activation)   (None, 111, 111, 32) 0           block1_conv1_bn[0][0]            
______________________________________________________________________________________________

Epoch 1/500
157/157 - 84s - loss: 2.8423 - accuracy: 0.1615 - val_loss: 2.3800 - val_accuracy: 0.3318
Epoch 2/500
157/157 - 75s - loss: 1.9387 - accuracy: 0.4126 - val_loss: 1.7831 - val_accuracy: 0.4450
Epoch 3/500
157/157 - 75s - loss: 1.5566 - accuracy: 0.5230 - val_loss: 1.7244 - val_accuracy: 0.4607
Epoch 4/500
157/157 - 75s - loss: 1.3028 - accuracy: 0.5966 - val_loss: 1.6289 - val_accuracy: 0.4906
Epoch 5/500
157/157 - 75s - loss: 1.0932 - accuracy: 0.6565 - val_loss: 1.6333 - val_accuracy: 0.5157
Epoch 6/500
157/157 - 75s - loss: 0.9326 - accuracy: 0.7133 - val_loss: 1.6645 - val_accuracy: 0.5236
Epoch 7/500
157/157 - 75s - loss: 0.7909 - accuracy: 0.7545 - val_loss: 1.8530 - val_accuracy: 0.5220
Epoch 8/500
157/157 - 76s - loss: 0.5688 - accuracy: 0.8141 - val_loss: 1.5707 - val_accuracy: 0.5676
Epoch 9/500
157/157 - 75s - loss: 0.4215 - accuracy: 0.8653 - val_loss: 1.6083 - val_accuracy: 0.5660
Epoch 10/500
157/157 - 76s - loss: 0.3530 - accuracy: 0.8880 - val_loss: 1.6940 - 

C:\Users\laptop\AppData\Roaming\Python\Python38\site-packages\keras\utils\generic_utils.py:494: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  warnings.warn('Custom mask layers require a config and must override '


Model saved.
Training Time: 2050.2985994815826


Testing:
40/40 [==============================] - 5s 100ms/step - loss: 1.8807 - accuracy: 0.5912




NameError: name 'predIdxs' is not defined

In [19]:
from tensorflow.keras.preprocessing import image_dataset_from_directory
input_path = 'Data/'
train_data = image_dataset_from_directory(directory=input_path,
                                          batch_size=16,
                                          image_size=(224, 224))
key = train_data.class_names

Found 3137 files belonging to 23 classes.


In [20]:
def read_directory():
    data_filenames = []
    data_labels = []
    for i in range (len(key)):
        for filename in os.listdir(input_path + key[i]):
                data_filenames.append(input_path +key[i]+'/' + filename)
                data_labels.append(i)
  
    return  data_filenames, data_labels

In [21]:
def build_test_tfrecord(train_filenames, train_labels):  # Generate TFRecord of training set 
    with tf.io.TFRecordWriter(test_tfrecord)as writer:
        for filename, label in zip(train_filenames, train_labels):
            image = open(filename, 'rb').read()

            feature = {
                'image': tf.train.Feature(bytes_list=tf.train.BytesList(value=[image])),  # img > Bytes
                'label': tf.train.Feature(int64_list=tf.train.Int64List(value=[label]))  # label > Int
            }

            example = tf.train.Example(features=tf.train.Features(feature=feature))
            writer.write(example.SerializeToString())

In [22]:
def _parse_example(example_string):
    feature_description = {
        'image': tf.io.FixedLenFeature([], tf.string),
        'label': tf.io.FixedLenFeature([], tf.int64),
    }

    feature_dict = tf.io.parse_single_example(example_string, feature_description)
    feature_dict['image'] = tf.io.decode_png(feature_dict['image'], channels=3)
    feature_dict['image'] = tf.image.resize(feature_dict['image'], [img_size, img_size]) / 255.0
    return feature_dict['image'], feature_dict['label']

In [23]:
def get_test_dataset(test_tfrecord):
    raw_test_dataset = tf.data.TFRecordDataset(test_tfrecord)
    test_dataset = raw_test_dataset.map(_parse_example)

    return test_dataset

In [24]:
def data_Preprocessing( test_dataset):
    test_dataset = test_dataset.batch(batch_size)
    test_dataset = test_dataset.prefetch(tf.data.experimental.AUTOTUNE)

    return  test_dataset

In [25]:
def InceptionResNetV2_model():

    engine = tf.keras.applications.Xception(
        # Freezing the weights of the top layer in the InceptionResNetV2 pre-traiined model
        include_top = False,

        # Use Imagenet weights
        weights = 'imagenet',

        # Define input shape to 224x224x3
        input_shape = (img_size , img_size , 3),

    )

    x = tf.keras.layers.GlobalAveragePooling2D(name = 'avg_pool')(engine.output)
    x =Dropout(0.85)(x)
    out = tf.keras.layers.Dense(len(key), activation = 'softmax', name = 'dense_output')(x)


    # Build the Keras model

    model = tf.keras.models.Model(inputs = engine.input, outputs = out)
    # Compile the model

    model.compile(
        # Set optimizer to Adam(0.0001)
        optimizer = tf.keras.optimizers.Adam(learning_rate= 3e-4),
        #optimizer= SGD(lr=0.0001, momentum=0.9),
        # Set loss to binary crossentropy
        loss = tf.keras.losses.sparse_categorical_crossentropy,

        # Set metrics to accuracy
        metrics = ['accuracy']
    )


    return model

In [26]:
def test(test_labels):
    test_labels = np.array(test_labels)

    from tensorflow import keras
    print('Testing:')
    mod = InceptionResNetV2_model()
    mod.load_weights('layer22-wskindiseases1.h5')
    mod.evaluate(test_dataset)
    
    prob = mod.predict(test_dataset)
    predIdxs = np.argmax(prob, axis=1) 


    print('\n')
    print(classification_report(test_labels, predIdxs,target_names = key, digits=5))
    return test_labels, predIdxs, prob, mod

In [27]:
if __name__ == "__main__":
    
    test_filenames, test_labels = read_directory()
    test_tfrecord = 'XRay_test.tfrecords'
    build_test_tfrecord(test_filenames, test_labels)


    test_dataset = get_test_dataset(test_tfrecord)



    test_dataset = data_Preprocessing(test_dataset) 

    test_labels, predIdxs,prob, Model = test(test_labels)
    


Testing:
197/197 [==============================] - 23s 98ms/step - loss: 0.4467 - accuracy: 0.8852.4215 - accuracy:


                           precision    recall  f1-score   support

          Acne Keloidalis    0.97807   0.97807   0.97807       228
          Alopecia Areata    0.94872   0.97368   0.96104       342
         Anagen effluvium    0.75524   0.76596   0.76056       141
  Androgenic Alopecia Men    0.90698   0.92857   0.91765        84
Androgenic Alopecia Women    0.81481   0.88000   0.84615       100
    Congenital pili torti    1.00000   0.95833   0.97872        24
            Discoid Lupus    0.77540   0.72864   0.75130       199
    Dissecting Cellulitis    0.70339   0.70339   0.70339       118
   Folliculitis Decalvans    0.85366   0.80000   0.82596       175
                Hirsutism    0.97973   0.99315   0.98639       146
        Hot Comb Alopecia    0.82443   0.90000   0.86056       120
          Ingrown Eyelash    0.98990   0.98000   0.98492       100
      Lic

In [28]:
def dermai_accuracy(prob, prob_Desicon):
    dermai = []
    for i in range(len(prob)):
        catogery =[]
        for j in range(len(prob[i])):
            if prob[i][j] > prob_Desicon:
                catogery.append(j)
        dermai.append(catogery)
    Positive = []
    for i in range(len(dermai)):
        if test_labels[i] in dermai[i]:
            Positive.append(1)
    accuracy = (len(Positive)/len(test_labels))*100  
    return  accuracy,dermai

In [29]:
accuracy,dermai = dermai_accuracy(prob, 0.10)

In [36]:
accuracy

88.25065274151436

In [37]:
v2=0
v3=0
vn = 0
for i in range (len(dermai)):
    if len(dermai[i])==1:
        pass
    elif len(dermai[i])==2:
        v2+=1
    elif len(dermai[i])==3:
        v3+=1
    else:
        vn+=1
print((v2/len(dermai))*100)
print((v3/len(dermai))*100)
print((vn/len(dermai))*100)
print((v2/len(dermai))*100+(v3/len(dermai))*100+(vn/len(dermai))*100)

8.093994778067886
2.8720626631853787
0.39164490861618795
11.357702349869452


In [38]:
(v2/len(dermai))*100

8.093994778067886

In [39]:
(v3/len(dermai))*100

2.8720626631853787

In [38]:
n_class = 20
dic = {}
values = []
index = []
for i in range(len(predIdxs)):
          if test_labels[i] == n_class: 
                if predIdxs[i] not in dic:
                        dic[predIdxs[i]]=1
                        index.append(i)
                        values.append(predIdxs[i])
                else:
                    dic[predIdxs[i]]+=1
                    index.append(i)
                    values.append(predIdxs[i])

In [39]:
dic

{20: 61, 15: 1, 16: 3, 2: 3, 5: 3, 17: 2, 4: 10, 10: 1}